In [1]:
# https://dagshub.com/kanishka-rani-2005/RenalTumor.mlflow

In [2]:
# import dagshub
# dagshub.init(repo_owner='kanishka-rani-2005', repo_name='RenalTumor', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [3]:
import os
%pwd


'e:\\RenalTumor\\research'

In [4]:
os.chdir('../')
%pwd

'e:\\RenalTumor'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/kanishka-rani-2005/RenalTumor.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'kanishka-rani-2005'
os.environ['MLFLOW_TRACKING_PASSWORD']='98da3ea8cf84b05159cc637c7414edb3af83bbe0'

In [6]:
import tensorflow as tf
model=tf.keras.models.load_model('artifacts\\training\\model.keras')


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_url:str
    params_image_size:list
    params_batch_size:int

In [8]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml, create_directories ,save_json



In [9]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.keras',
            training_data='artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            all_params=self.params, 
            mlflow_url='https://dagshub.com/kanishka-rani-2005/RenalTumor.mlflow',
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE 
        )
        return eval_config

In [10]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from pathlib import Path

e:\RenalTumor\venv\Lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self.train_valid_generator()
        self.score=self.model.evaluate(self.valid_generator)
        self.save_score()
        self.model.save("model.keras")

    
    def save_score(self):
        scores={'loss': self.score[0], 'accuracy':self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_score=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    'loss':self.score[0],
                    'accuracy':self.score[1]
                }
            )
            if tracking_url_type_score != 'file':
            # Log the single model.keras file
                mlflow.log_artifact("model.keras", artifact_path="model")
            else:
                mlflow.log_artifact("model.keras", artifact_path="model")

In [12]:
try:
    config=ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e


[Fri, 04 Jul 2025 20:14:12: INFO: common: Loading Yaml file succesfully.]
[Fri, 04 Jul 2025 20:14:12: INFO: common: Loading Yaml file succesfully.]
[Fri, 04 Jul 2025 20:14:12: INFO: common: Directory created at: artifacts]
Found 2487 images belonging to 4 classes.


e:\RenalTumor\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 233s 3s/step - accuracy: 0.6282 - loss: 5.9766
[Fri, 04 Jul 2025 20:18:06: INFO: common: Json file saved at: scores.json]
[Fri, 04 Jul 2025 20:18:07: INFO: common: Json file saved at: scores.json]
[Fri, 04 Jul 2025 21:03:19: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host')': /kanishka-rani-2005/RenalTumor.mlflow/api/2.0/mlflow-artifacts/artifacts/ec968b1b75bb48548e56182e725c96ea/e109637d21d8490099d36f1f5e2241e8/artifacts/model/model.keras]
[Fri, 04 Jul 2025 21:03:23: WARNING: connectionpool: Retrying (Retry(total=3, connect=4, read=4, redirect=5, status=5)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000028F03FD4450>: Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /kanishka-rani-2005/RenalTumor.mlflow/api/2.0/mlflow-artif